<a href="https://colab.research.google.com/github/UnmilonGeophysics/Image_classification-NVIDIA-Course-Project/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')# Initialize VGG16 with transfer learning




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ! pip install kaggle

In [24]:
#from google.colab import files
# files.upload()

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification

100% 3.58G/3.58G [01:07<00:00, 82.4MB/s]
100% 3.58G/3.58G [01:07<00:00, 57.4MB/s]


In [8]:
mkdir data

In [9]:
! unzip fruits-fresh-and-rotten-for-classification.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.09 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.18 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.24 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.37 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.51 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.02 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.12 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.21 PM.png  
  inflating: data/dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.31 PM.png  
  inflating: data/dataset/train

In [10]:
train_path='/content/data/dataset/train'
test_path='/content/data/dataset/test'

In [ ]:
#Downloading the pretrained model vggg16

In [11]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [12]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
# add a global spatial average pooling layer
x = base_model.output

x = keras.layers.GlobalAveragePooling2D()(x)
# and a dense layer
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(6, activation='softmax')(x)

# this is the model we will train
model = keras.Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), metrics=['accuracy'], loss='categorical_crossentropy')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
train_datagen = ImageDataGenerator(rescale = 1./255,
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
#  load and iterate training dataset
train_it = train_datagen.flow_from_directory(train_path, 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical",
                                       batch_size=32)
# load and iterate validation dataset
test_it = test_datagen.flow_from_directory(test_path, 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical",
                                      batch_size=32)
                                      

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


In [19]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=len(train_it),
          validation_steps=len(test_it),
          epochs=10)

Epoch 1/10
341/341 [==============================] - 257s 635ms/step - loss: 0.5309 - accuracy: 0.8153 - val_loss: 0.2380 - val_accuracy: 0.9166
Epoch 2/10
341/341 [==============================] - 204s 599ms/step - loss: 0.2178 - accuracy: 0.9251 - val_loss: 0.2556 - val_accuracy: 0.9036
Epoch 3/10
341/341 [==============================] - 206s 604ms/step - loss: 0.1622 - accuracy: 0.9440 - val_loss: 0.1560 - val_accuracy: 0.9429
Epoch 4/10
341/341 [==============================] - 203s 595ms/step - loss: 0.1379 - accuracy: 0.9496 - val_loss: 0.2377 - val_accuracy: 0.9051
Epoch 5/10
341/341 [==============================] - 206s 603ms/step - loss: 0.1312 - accuracy: 0.9524 - val_loss: 0.1732 - val_accuracy: 0.9296
Epoch 6/10
341/341 [==============================] - 204s 598ms/step - loss: 0.0999 - accuracy: 0.9627 - val_loss: 0.1386 - val_accuracy: 0.9489
Epoch 7/10
341/341 [==============================] - 205s 602ms/step - loss: 0.0997 - accuracy: 0.9628 - val_loss: 0.1693 -

In [23]:
model.evaluate(test_it)

85/85 [==============================] - 21s 241ms/step - loss: 0.1384 - accuracy: 0.9429


[0.13837432861328125, 0.9429206848144531]